In [2]:
import tensorflow as tf
import numpy as np
import time
import sys

In [3]:
class HashModP64(tf.Module):
    """Universal hash for integers via affine-mod-prime transformation."""
    def __init__(
            self,
            num_hashes,
            prime = 2**31 - 1,
            seed = None
            ):
        """Hash function constructor.

        Arguments:
            num_hashes: Integer. Number of hash values to compute for each
                input.
            prime: Integer. Large prime number used to compute the modulus.
            seed: Integer. Seed for repeatable hashing across class instances.
        """
        self._num_hashes = num_hashes
        self._prime = prime
        self._seed = seed
        initializer_a = tf.random_uniform_initializer(
            minval=1, maxval=self._prime - 1, seed=self._seed)
        initializer_b = tf.random_uniform_initializer(
            minval=0, maxval=self._prime - 1, seed=self._seed)
        self._a = tf.Variable(
            initializer_a(shape=[self._num_hashes], dtype=tf.int64))
        self._b = tf.Variable(
            initializer_b(shape=[self._num_hashes], dtype=tf.int64))

    @tf.function(
        input_signature=[tf.TensorSpec(shape=None, dtype=tf.int64)])
    def hash(self, x):
        """Calculates hashes for input tensor x.

        Arguments:
            x: tf.Tensor of rank R, type tf.int64 and arbitrary shape
            [N0, N1, ... NR].

        Returns:
            A rank R+1 tf.Tensor of shape [N0, N1, ... NR, num_hashes] that
            contains tf.int64 hash codes.
        """
        affine = tf.tensordot(x, self._a, axes=0) + self._b
        return tf.math.floormod(affine, self._prime)

class HashBitMix64(tf.Module):
    """Universal hash for integers via mixing with bitwise operations."""
    def __init__(
            self, 
            num_hashes,
            seed = None
            ):
        """Hash function constructor.

        Arguments:
            num_hashes: Integer. Number of hash values to compute for each
                input.
            seed: Integer. Seed for repeatable hashing across class instances.
        """
        if num_hashes < 1:
            raise ValueError("num_hashes must be >= 1 but is %s" % num_hashes)

        self._num_hashes = num_hashes
        self._seed = seed
        self._c1 = tf.convert_to_tensor(0xbf58476d1ce4e5b9, dtype=tf.uint64)
        self._c2 = tf.convert_to_tensor(0x94d049bb133111eb, dtype=tf.uint64)
        initializer_xor = tf.random_uniform_initializer(
            minval=1, maxval=2**31 - 1, seed=self._seed)
        self._xor_constants = tf.Variable(
            initializer_xor(shape=[self._num_hashes], dtype=tf.int64))

    @tf.function(
        input_signature=[tf.TensorSpec(shape=None, dtype=tf.int64)])
    def hash(self, x):
        """Calculates hashes for input tensor x.

        Arguments:
            x: tf.Tensor of rank R, type tf.int64 and arbitrary shape
            [N0, N1, ... NR].

        Returns:
            A rank R+1 tf.Tensor of shape [N0, N1, ... NR, num_hashes] that
            contains tf.int64 hash codes.
        """
        x = tf.stack(
            [tf.bitwise.bitwise_xor(x, self._xor_constants[i])
            for i in range(self._num_hashes)], axis=-1)
        # Use uint64 mixer (constants found via annealing by David Stafford).
        x = tf.bitcast(x, tf.uint64)  # Does not copy data.
        x = tf.bitwise.bitwise_xor(x, tf.bitwise.right_shift(x, 30))
        x = x * self._c1
        x = tf.bitwise.bitwise_xor(x, tf.bitwise.right_shift(x, 27))
        x = x * self._c2
        x = tf.bitwise.bitwise_xor(x, tf.bitwise.right_shift(x, 31))
        return tf.bitcast(x, tf.int64)


In [4]:
class HashModP32(tf.Module):
    """Universal hash for integers via affine-mod-prime transformation."""
    def __init__(
            self,
            num_hashes,
            prime = 2**31 - 1,
            seed = None
            ):
        """Hash function constructor.

        Arguments:
            num_hashes: Integer. Number of hash values to compute for each
                input.
            prime: Integer. Large prime number used to compute the modulus.
            seed: Integer. Seed for repeatable hashing across class instances.
        """
        self._num_hashes = num_hashes
        self._prime = prime
        self._seed = seed
        initializer_a = tf.random_uniform_initializer(
            minval=1, maxval=self._prime - 1, seed=self._seed)
        initializer_b = tf.random_uniform_initializer(
            minval=0, maxval=self._prime - 1, seed=self._seed)
        self._a = tf.Variable(
            initializer_a(shape=[self._num_hashes], dtype=tf.int32))
        self._b = tf.Variable(
            initializer_b(shape=[self._num_hashes], dtype=tf.int32))

    @tf.function(
        input_signature=[tf.TensorSpec(shape=None, dtype=tf.int32)])
    def hash(self, x):
        """Calculates hashes for input tensor x.

        Arguments:
            x: tf.Tensor of rank R, type tf.int32 and arbitrary shape
            [N0, N1, ... NR].

        Returns:
            A rank R+1 tf.Tensor of shape [N0, N1, ... NR, num_hashes] that
            contains tf.int64 hash codes.
        """
        affine = tf.tensordot(x, self._a, axes=0) + self._b
        return tf.math.floormod(affine, self._prime)

class HashBitMix32(tf.Module):
    """Universal hash for integers via mixing with bitwise operations."""
    def __init__(
            self, 
            num_hashes,
            seed = None
            ):
        """Hash function constructor.

        Arguments:
            num_hashes: Integer. Number of hash values to compute for each
                input.
            seed: Integer. Seed for repeatable hashing across class instances.
        """
        if num_hashes < 1:
            raise ValueError("num_hashes must be >= 1 but is %s" % num_hashes)

        self._num_hashes = num_hashes
        self._seed = seed
        self._c1 = tf.convert_to_tensor(0x85ebca6b, dtype=tf.uint32)
        self._c2 = tf.convert_to_tensor(0xc2b2ae35, dtype=tf.uint32)
        initializer_xor = tf.random_uniform_initializer(
            minval=1, maxval=2**31 - 1, seed=self._seed)
        self._xor_constants = tf.Variable(
            initializer_xor(shape=[self._num_hashes], dtype=tf.int32))

    @tf.function(
        input_signature=[tf.TensorSpec(shape=None, dtype=tf.int32)])
    def hash(self, x):
        """Calculates hashes for input tensor x.

        Arguments:
            x: tf.Tensor of rank R, type tf.int32 and arbitrary shape
            [N0, N1, ... NR].

        Returns:
            A rank R+1 tf.Tensor of shape [N0, N1, ... NR, num_hashes] that
            contains tf.int64 hash codes.
        """
        x = tf.stack(
            [tf.bitwise.bitwise_xor(x, self._xor_constants[i])
            for i in range(self._num_hashes)], axis=-1)
        # Use uint64 mixer (constants found via annealing by David Stafford).
        x = tf.bitcast(x, tf.uint32)  # Does not copy data.
        x = tf.bitwise.bitwise_xor(x, tf.bitwise.right_shift(x, 16))
        x = x * self._c1
        x = tf.bitwise.bitwise_xor(x, tf.bitwise.right_shift(x, 13))
        x = x * self._c2
        x = tf.bitwise.bitwise_xor(x, tf.bitwise.right_shift(x, 16))
        return tf.bitcast(x, tf.int32)

In [5]:
def run_benchmark64(device,
                  num_hashes=100,
                  batch_size=512,
                  num_tokens=1000,
                  experiment_reps=500):
    with tf.device(device):
        input_shape = [batch_size, num_tokens]
        initializer = tf.random_uniform_initializer(
            minval=int(-1e6), maxval=int(1e6))
        x = [tf.Variable(initializer(shape=input_shape, dtype=tf.int64))
             for _ in range(experiment_reps)]
        h1 = HashBitMix64(num_hashes)
        h2 = HashModP64(num_hashes)
        h1_times = []
        for xi in x:
            t0 = time.time()
            y = h1.hash(xi)
            t1 = time.time()
            h1_times.append(t1-t0)
        h2_times = []
        for xi in x:
            t0 = time.time()
            y = h2.hash(xi)
            t1 = time.time()
            h2_times.append(t1-t0)
    return (h1_times, h2_times)

def run_benchmark32(device,
                  num_hashes=100,
                  batch_size=512,
                  num_tokens=1000,
                  experiment_reps=500):
    with tf.device(device):
        input_shape = [batch_size, num_tokens]
        initializer = tf.random_uniform_initializer(
            minval=int(-1e6), maxval=int(1e6))
        x = [tf.Variable(initializer(shape=input_shape, dtype=tf.int32))
             for _ in range(experiment_reps)]
        h1 = HashBitMix32(num_hashes)
        h2 = HashModP32(num_hashes)
        h1_times = []
        for xi in x:
            t0 = time.time()
            y = h1.hash(xi)
            t1 = time.time()
            h1_times.append(t1-t0)
        h2_times = []
        for xi in x:
            t0 = time.time()
            y = h2.hash(xi)
            t1 = time.time()
            h2_times.append(t1-t0)
    return (h1_times, h2_times)

In [6]:
h1_times_cpu_64, h2_times_cpu_64 = run_benchmark64('/cpu:0')
print("On CPU (64-bit)")
print("HashBitMix: ",np.mean(h1_times_cpu_64), np.std(h1_times_cpu_64))
print("HashModP: ",np.mean(h2_times_cpu_64), np.std(h2_times_cpu_64))
sys.stdout.flush()

h1_times_gpu_64, h2_times_gpu_64 = run_benchmark64('/device:GPU:0')
print("On GPU (64-bit)")
print("HashBitMix: ",np.mean(h1_times_gpu_64), np.std(h1_times_gpu_64))
print("HashModP: ",np.mean(h2_times_gpu_64), np.std(h2_times_gpu_64))
sys.stdout.flush()

h1_times_cpu_32, h2_times_cpu_32 = run_benchmark32('/cpu:0')
print("On CPU (32-bit)")
print("HashBitMix: ",np.mean(h1_times_cpu_32), np.std(h1_times_cpu_32))
print("HashModP: ",np.mean(h2_times_cpu_32), np.std(h2_times_cpu_32))
sys.stdout.flush()

h1_times_gpu_32, h2_times_gpu_32 = run_benchmark32('/device:GPU:0')
print("On GPU (32-bit)")
print("HashBitMix: ",np.mean(h1_times_gpu_32), np.std(h1_times_gpu_32))
print("HashModP: ",np.mean(h2_times_gpu_32), np.std(h2_times_gpu_32))
sys.stdout.flush()

On CPU (64-bit)
HashBitMix:  1.2728994584083557 0.0606345520431441
HashModP:  0.822510507106781 0.019799892577717637
On GPU (64-bit)
HashBitMix:  0.09299386167526245 0.023977337576366914
HashModP:  0.8131655917167664 0.026527320797214995
On CPU (32-bit)
HashBitMix:  0.8894901666641235 0.03794307909790895
HashModP:  0.3682930154800415 0.010422571491485655
On GPU (32-bit)
HashBitMix:  0.567615369796753 0.04233825338727441
HashModP:  0.35730077362060547 0.00836473357131244
